In [ ]:
from snowflake.snowpark import Session
from snowflake.snowpark.functions import col
from snowflake.ml.modeling.preprocessing import OneHotEncoder, StandardScaler
from snowflake.ml.modeling import LogisticRegression
from snowflake.ml.model import save_model
import pandas as pd

session = Session.builder.configs(connection_params).create()

df = session.table("CUSTOMER_CHURN")

# Feature & label split
features = ["age", "city", "plan", "monthly_usage", "complaints", "payments_missed", "tenure"]
label = "churn_label"

# Encoding categorical features
encoder = OneHotEncoder(input_cols=["city", "plan"])
df_encoded = encoder.fit(df).transform(df)

# Scale numeric columns
scaler = StandardScaler(input_cols=["age", "monthly_usage", "complaints", "payments_missed", "tenure"])
df_scaled = scaler.fit(df_encoded).transform(df_encoded)

# Train/test split
train_df, test_df = df_scaled.randomSplit([0.8, 0.2])

# Train model
model = LogisticRegression(label_col=label)
fitted = model.fit(train_df)

# Save model
save_model(fitted, session, stage_location="@model_stage/churn_model")